In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import time, json, datetime
from tqdm import tqdm
import sys, getopt

import time
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
class DeepFM(nn.Module):
    def __init__(self, cate_fea_nuniqs, nume_fea_size=0, emb_size=8,
                 hid_dims=[256, 128], num_classes=1, dropout=[0.2, 0.2]):
        """
        cate_fea_nuniqs: 类别特征的唯一值个数列表，也就是每个类别特征的vocab_size所组成的列表
        nume_fea_size: 数值特征的个数，该模型会考虑到输入全为类别型，即没有数值特征的情况
        """
        super().__init__()
        self.cate_fea_size = len(cate_fea_nuniqs)
        self.nume_fea_size = nume_fea_size

        """FM部分"""
        # 一阶
        if self.nume_fea_size != 0:
            self.fm_1st_order_dense = nn.Linear(self.nume_fea_size, 1)  # 数值特征的一阶表示
        self.fm_1st_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, 1) for voc_size in cate_fea_nuniqs])  # 类别特征的一阶表示

        # 二阶
        self.fm_2nd_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, emb_size) for voc_size in cate_fea_nuniqs])  # 类别特征的二阶表示

        """DNN部分"""
        self.all_dims = [self.cate_fea_size * emb_size] + hid_dims
        self.dense_linear = nn.Linear(self.nume_fea_size, self.cate_fea_size * emb_size)  # 数值特征的维度变换到FM输出维度一致
        self.relu = nn.ReLU()
        # for DNN
        for i in range(1, len(self.all_dims)):
            setattr(self, 'linear_' + str(i), nn.Linear(self.all_dims[i - 1], self.all_dims[i]))
            setattr(self, 'batchNorm_' + str(i), nn.BatchNorm1d(self.all_dims[i]))
            setattr(self, 'activation_' + str(i), nn.ReLU())
            setattr(self, 'dropout_' + str(i), nn.Dropout(dropout[i - 1]))
        # for output
        self.dnn_linear = nn.Linear(hid_dims[-1], num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X_sparse, X_dense=None):
        """
        X_sparse: 类别型特征输入  [bs, cate_fea_size]
        X_dense: 数值型特征输入（可能没有）  [bs, dense_fea_size]
        """

        """FM 一阶部分"""
        fm_1st_sparse_res = [emb(X_sparse[:, i].unsqueeze(1)).view(-1, 1)
                             for i, emb in enumerate(self.fm_1st_order_sparse_emb)]
        fm_1st_sparse_res = torch.cat(fm_1st_sparse_res, dim=1)  # [bs, cate_fea_size]
        fm_1st_sparse_res = torch.sum(fm_1st_sparse_res, 1, keepdim=True)  # [bs, 1]

        if X_dense is not None:
            fm_1st_dense_res = self.fm_1st_order_dense(X_dense)
            fm_1st_part = fm_1st_sparse_res + fm_1st_dense_res
        else:
            fm_1st_part = fm_1st_sparse_res  # [bs, 1]

        """FM 二阶部分"""
        fm_2nd_order_res = [emb(X_sparse[:, i].unsqueeze(1)) for i, emb in enumerate(self.fm_2nd_order_sparse_emb)]
        fm_2nd_concat_1d = torch.cat(fm_2nd_order_res, dim=1)  # [bs, n, emb_size]  n为类别型特征个数(cate_fea_size)

        # 先求和再平方
        sum_embed = torch.sum(fm_2nd_concat_1d, 1)  # [bs, emb_size]
        square_sum_embed = sum_embed * sum_embed  # [bs, emb_size]
        # 先平方再求和
        square_embed = fm_2nd_concat_1d * fm_2nd_concat_1d  # [bs, n, emb_size]
        sum_square_embed = torch.sum(square_embed, 1)  # [bs, emb_size]
        # 相减除以2
        sub = square_sum_embed - sum_square_embed
        sub = sub * 0.5  # [bs, emb_size]

        fm_2nd_part = torch.sum(sub, 1, keepdim=True)  # [bs, 1]

        """DNN部分"""
        dnn_out = torch.flatten(fm_2nd_concat_1d, 1)  # [bs, n * emb_size]

        if X_dense is not None:
            dense_out = self.relu(self.dense_linear(X_dense))  # [bs, n * emb_size]
            dnn_out = dnn_out + dense_out  # [bs, n * emb_size]

        for i in range(1, len(self.all_dims)):
            dnn_out = getattr(self, 'linear_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'batchNorm_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'activation_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'dropout_' + str(i))(dnn_out)

        dnn_out = self.dnn_linear(dnn_out)  # [bs, 1]
        out = fm_1st_part + fm_2nd_part + dnn_out  # [bs, 1]
        out = self.sigmoid(out)
        return out

In [4]:
def train_and_eval(model, train_loader, valid_loader, epochs, optimizer, loss_fcn, scheduler, device,ii):
    import time
    best_auc=0
    for _ in range(epochs):
        """训练部分"""
        model.train()
        print("Current lr : {}".format(optimizer.state_dict()['param_groups'][0]['lr']))
        write_log('Epoch: {}:{}'.format(ii,_))
        train_loss_sum = 0.0
        start_time = time.time()
        for idx, x in enumerate(train_loader):
            cate_fea, nume_fea, label = x[0], x[1], x[2]
            cate_fea, nume_fea, label = cate_fea.to(device), nume_fea.to(device), label.float().to(device)
            pred = model(cate_fea, nume_fea).view(-1)
            loss = loss_fcn(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.cpu().item()
            if (idx + 1) % 50 == 0 or (idx + 1) == len(train_loader):
                write_log("Epoch {:04d} | Step {:04d} / {} | Loss {:.4f} | Time {:.4f}".format(
                    _ + 1, idx + 1, len(train_loader), train_loss_sum / (idx + 1), time.time() - start_time))
        scheduler.step()
        """推断部分"""
        model.eval()
        with torch.no_grad():
            valid_labels, valid_preds = [], []
            for idx, x in tqdm(enumerate(valid_loader)):
                cate_fea, nume_fea, label = x[0], x[1], x[2]
                cate_fea, nume_fea = cate_fea.to(device), nume_fea.to(device)
                pred = model(cate_fea, nume_fea).reshape(-1).data.cpu().numpy().tolist()
                valid_preds.extend(pred)
                valid_labels.extend(label.cpu().numpy().tolist())
        cur_auc = roc_auc_score(valid_labels, valid_preds)
        if cur_auc > best_auc:
            best_auc = cur_auc
            import time
            end=time.time()
            torch.save(model.state_dict(), "../../data/wj/deepfm_best/deepfm_best10fen_"+str(round(best_auc,4))+"_"+str(ii)+"_"+str(_)+"_"+str(time.strftime('%m_%d_%H_%M_%S'))+".pth")
        write_log('Current AUC: %.6f, Best AUC: %.6f\n' % (cur_auc, best_auc))




# 定义日志（data文件夹下，同级目录新建一个data文件夹）
def write_log(w):
    file_name = '../../data/wj/' + datetime.date.today().strftime('%m%d') + "_{}.log".format("deepfm_10fen")
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info + '\n')
        
        

In [5]:
doc_feat = pd.read_pickle('../../data/wj/doc.pkl')
user_feat = pd.read_pickle('../../data/wj/user.pkl')
df_test_user_doc=pd.read_pickle('../../data/wj/df_test_user_doc.pkl')

sparse_features = ['userid', 'docid', 'network', 'hour', 'device', 'os', 'province',
                   'city', 'age', 'gender', 'category1st', 'category2nd',
                   'pub_date', 'keyword0', 'keyword1', 'keyword2', 'keyword3', 'keyword4']

dense_features = ['refresh', 'picnum',
                  'userid_click_mean','userid_click_count' ,'userid_duration_mean' ,'userid_picnum_mean',
                    'docid_click_mean','docid_click_count','docid_duration_mean','docid_picnum_mean',
                    'category1st_click_mean','category1st_click_count','category1st_duration_mean','category1st_picnum_mean',
                    'category2nd_click_mean','category2nd_click_count','category2nd_duration_mean','category2nd_picnum_mean',
                    'keyword0_click_mean','keyword0_click_count','keyword0_duration_mean','keyword0_picnum_mean']

cate_fea_nuniqs = []
cate_fea_nuniqs.append(user_feat['userid'].nunique() + 1)
cate_fea_nuniqs.append(doc_feat['docid'].nunique() + 1)
cate_fea_nuniqs.append(6)  # network
cate_fea_nuniqs.append(13)  # hour
cate_fea_nuniqs.append(user_feat['device'].nunique())
cate_fea_nuniqs.append(user_feat['os'].nunique())
cate_fea_nuniqs.append(user_feat['province'].nunique())
cate_fea_nuniqs.append(user_feat['city'].nunique())
cate_fea_nuniqs.append(user_feat['age'].nunique())
cate_fea_nuniqs.append(user_feat['gender'].nunique())
cate_fea_nuniqs.append(doc_feat['category1st'].nunique())
cate_fea_nuniqs.append(doc_feat['category2nd'].nunique())
cate_fea_nuniqs.append(doc_feat['pub_date'].nunique())
keyword_nunique = max(doc_feat['keyword0'].max(), doc_feat['keyword1'].max(), doc_feat['keyword2'].max()
                      , doc_feat['keyword3'].max(), doc_feat['keyword4'].max()) + 1
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)

device = torch.device('cuda:3') if torch.cuda.is_available() else torch.device('cpu')
loss_fcn = nn.BCELoss()
loss_fcn = loss_fcn.to(device)

In [7]:
import pandas as pd


len_=81671133
ii=-1
for i in range(0,len_,round(len_/10)):
    ii+=1
    right=i+round(len_/10)
    if right>=len_:
        break
    print(i,right)
    df_train_user_doc=pd.read_pickle('../../data/wj/df_train_user_doc_0_1_'+str(i)+'_'+str(right)+'.pkl')

    train, valid = train_test_split(df_train_user_doc, test_size=0.25, random_state=2021)
#         train_loader，valid_loader
    train_dataset = Data.TensorDataset(torch.LongTensor(train[sparse_features].values),
                                       torch.FloatTensor(train[dense_features].values),
                                       torch.FloatTensor(train['click'].values), )
    train_loader = Data.DataLoader(dataset=train_dataset, batch_size=4096, shuffle=True)
    valid_dataset = Data.TensorDataset(torch.LongTensor(valid[sparse_features].values),
                                       torch.FloatTensor(valid[dense_features].values),
                                       torch.FloatTensor(valid['click'].values))
    valid_loader = Data.DataLoader(dataset=valid_dataset, batch_size=4096, shuffle=False)
    
#         train
    print('train')
    model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
    epoch_=3
    train_and_eval(model, train_loader, valid_loader, epoch_, optimizer, loss_fcn, scheduler, device,ii)

0 8167113
train
Current lr : 0.005
05:15:42 : Epoch: 0:0
05:15:49 : Epoch 0001 | Step 0050 / 1496 | Loss 19.4318 | Time 7.4697
05:15:56 : Epoch 0001 | Step 0100 / 1496 | Loss 12.7530 | Time 14.2757
05:16:03 : Epoch 0001 | Step 0150 / 1496 | Loss 9.7791 | Time 21.1058
05:16:10 : Epoch 0001 | Step 0200 / 1496 | Loss 7.7866 | Time 27.9825
05:16:16 : Epoch 0001 | Step 0250 / 1496 | Loss 6.4300 | Time 34.7821
05:16:23 : Epoch 0001 | Step 0300 / 1496 | Loss 5.4767 | Time 41.7688
05:16:31 : Epoch 0001 | Step 0350 / 1496 | Loss 4.7764 | Time 49.3049
05:16:38 : Epoch 0001 | Step 0400 / 1496 | Loss 4.2411 | Time 56.2472
05:16:45 : Epoch 0001 | Step 0450 / 1496 | Loss 3.8194 | Time 63.0139
05:16:51 : Epoch 0001 | Step 0500 / 1496 | Loss 3.4793 | Time 69.7788
05:16:58 : Epoch 0001 | Step 0550 / 1496 | Loss 3.1985 | Time 76.5327
05:17:05 : Epoch 0001 | Step 0600 / 1496 | Loss 2.9640 | Time 83.0528
05:17:12 : Epoch 0001 | Step 0650 / 1496 | Loss 2.7651 | Time 89.9733
05:17:19 : Epoch 0001 | Step 070

499it [00:52,  9.50it/s]


05:20:13 : Current AUC: 0.769768, Best AUC: 0.769768

Current lr : 0.004
05:20:13 : Epoch: 0:1
05:20:22 : Epoch 0002 | Step 0050 / 1496 | Loss 0.3545 | Time 9.4639
05:20:31 : Epoch 0002 | Step 0100 / 1496 | Loss 0.3531 | Time 18.0725
05:20:39 : Epoch 0002 | Step 0150 / 1496 | Loss 0.3532 | Time 26.5879
05:20:48 : Epoch 0002 | Step 0200 / 1496 | Loss 0.3535 | Time 34.9804
05:20:56 : Epoch 0002 | Step 0250 / 1496 | Loss 0.3541 | Time 43.4489
05:21:05 : Epoch 0002 | Step 0300 / 1496 | Loss 0.3536 | Time 52.3763
05:21:14 : Epoch 0002 | Step 0350 / 1496 | Loss 0.3535 | Time 61.3451
05:21:23 : Epoch 0002 | Step 0400 / 1496 | Loss 0.3535 | Time 70.2232
05:21:32 : Epoch 0002 | Step 0450 / 1496 | Loss 0.3535 | Time 79.0309
05:21:40 : Epoch 0002 | Step 0500 / 1496 | Loss 0.3534 | Time 87.3800
05:21:48 : Epoch 0002 | Step 0550 / 1496 | Loss 0.3533 | Time 95.0709
05:21:55 : Epoch 0002 | Step 0600 / 1496 | Loss 0.3533 | Time 102.5689
05:22:03 : Epoch 0002 | Step 0650 / 1496 | Loss 0.3534 | Time 110

499it [00:47, 10.49it/s]


05:25:06 : Current AUC: 0.770536, Best AUC: 0.770536

Current lr : 0.0032
05:25:06 : Epoch: 0:2
05:25:14 : Epoch 0003 | Step 0050 / 1496 | Loss 0.3529 | Time 8.3746
05:25:22 : Epoch 0003 | Step 0100 / 1496 | Loss 0.3531 | Time 16.1130
05:25:30 : Epoch 0003 | Step 0150 / 1496 | Loss 0.3531 | Time 24.0386
05:25:37 : Epoch 0003 | Step 0200 / 1496 | Loss 0.3533 | Time 31.0978
05:25:45 : Epoch 0003 | Step 0250 / 1496 | Loss 0.3531 | Time 39.1073
05:25:54 : Epoch 0003 | Step 0300 / 1496 | Loss 0.3528 | Time 47.9722
05:26:01 : Epoch 0003 | Step 0350 / 1496 | Loss 0.3523 | Time 55.2814
05:26:09 : Epoch 0003 | Step 0400 / 1496 | Loss 0.3520 | Time 62.8942
05:26:16 : Epoch 0003 | Step 0450 / 1496 | Loss 0.3521 | Time 70.4227
05:26:24 : Epoch 0003 | Step 0500 / 1496 | Loss 0.3522 | Time 78.4696
05:26:33 : Epoch 0003 | Step 0550 / 1496 | Loss 0.3522 | Time 87.0956
05:26:40 : Epoch 0003 | Step 0600 / 1496 | Loss 0.3522 | Time 94.2300
05:26:48 : Epoch 0003 | Step 0650 / 1496 | Loss 0.3523 | Time 102

...

07:31:57 : Current AUC: 0.766923, Best AUC: 0.766923

Current lr : 0.004
07:31:57 : Epoch: 9:1
07:32:06 : Epoch 0002 | Step 0050 / 1496 | Loss 0.3537 | Time 8.9504
07:32:14 : Epoch 0002 | Step 0100 / 1496 | Loss 0.3537 | Time 16.7981
07:32:21 : Epoch 0002 | Step 0150 / 1496 | Loss 0.3534 | Time 24.2786
07:32:29 : Epoch 0002 | Step 0200 / 1496 | Loss 0.3535 | Time 32.3073
07:32:37 : Epoch 0002 | Step 0250 / 1496 | Loss 0.3536 | Time 40.4517
07:32:46 : Epoch 0002 | Step 0300 / 1496 | Loss 0.3534 | Time 49.3341
07:32:54 : Epoch 0002 | Step 0350 / 1496 | Loss 0.3531 | Time 57.0527
07:33:03 : Epoch 0002 | Step 0400 / 1496 | Loss 0.3533 | Time 65.7053
07:33:10 : Epoch 0002 | Step 0450 / 1496 | Loss 0.3530 | Time 73.4220
07:33:18 : Epoch 0002 | Step 0500 / 1496 | Loss 0.3531 | Time 81.4544
07:33:26 : Epoch 0002 | Step 0550 / 1496 | Loss 0.3531 | Time 88.9348
07:33:33 : Epoch 0002 | Step 0600 / 1496 | Loss 0.3531 | Time 96.0902
07:33:40 : Epoch 0002 | Step 0650 / 1496 | Loss 0.3532 | Time 103.

499it [00:48, 10.29it/s]


07:36:40 : Current AUC: 0.771042, Best AUC: 0.771042

Current lr : 0.0032
07:36:40 : Epoch: 9:2
07:36:49 : Epoch 0003 | Step 0050 / 1496 | Loss 0.3525 | Time 9.1730
07:36:58 : Epoch 0003 | Step 0100 / 1496 | Loss 0.3520 | Time 17.9612
07:37:04 : Epoch 0003 | Step 0150 / 1496 | Loss 0.3525 | Time 24.6272
07:37:12 : Epoch 0003 | Step 0200 / 1496 | Loss 0.3523 | Time 32.0785
07:37:20 : Epoch 0003 | Step 0250 / 1496 | Loss 0.3525 | Time 40.2083
07:37:29 : Epoch 0003 | Step 0300 / 1496 | Loss 0.3524 | Time 48.9276
07:37:36 : Epoch 0003 | Step 0350 / 1496 | Loss 0.3524 | Time 56.4963
07:37:44 : Epoch 0003 | Step 0400 / 1496 | Loss 0.3526 | Time 63.8727
07:37:52 : Epoch 0003 | Step 0450 / 1496 | Loss 0.3526 | Time 72.0446
07:38:00 : Epoch 0003 | Step 0500 / 1496 | Loss 0.3526 | Time 80.1169
07:38:09 : Epoch 0003 | Step 0550 / 1496 | Loss 0.3529 | Time 88.8308
07:38:17 : Epoch 0003 | Step 0600 / 1496 | Loss 0.3529 | Time 96.8925
07:38:25 : Epoch 0003 | Step 0650 / 1496 | Loss 0.3528 | Time 105

499it [00:45, 10.88it/s]


07:41:30 : Current AUC: 0.772897, Best AUC: 0.772897



In [38]:
torch.save(model.state_dict(), "../../data/wj/deepfm_best_"+str(0.7791)+"_"+str(1)+"_"+str(9)+".pth")

In [22]:
# df_train_user_doc.to_pickle('../../data/wj/df_train_user_doc_0.1.pkl')

In [5]:
# model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
# model.load_state_dict(torch.load('../../data/wj/deepfm_best_0.7739082162724398_1639060823.4660978.pth'))
# model.to(device)
# epoch = 5
# train_and_eval(model, train_loader, valid_loader, epoch, optimizer, loss_fcn, scheduler, device)